### In this notebook we will implements a classification model & perform cosine similarity  

In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import pickle
from numpy import dot
from numpy.linalg import norm

In [2]:
# reading the stop words list with pickle
with open (r'C:\Users\HP\OneDrive\Desktop\Diagnosis\Notebook\current medical diagnosis.ob', 'rb') as fp:
    domain_stop_word = pickle.load(fp)

In [6]:
# read data file
file_path = r'C:\Users\HP\OneDrive\Desktop\Diagnosis\Notebook\diseases_with_description1.csv'
df = pd.read_csv(file_path)
print(df.head())

                                         Description          D_Name
0  cancer, tumor, survival, carcinoma, prostate, ...          Cancer
1  artery, seizure, hg, study, kidney, weakness, ...        Diabetes
2  seizure, weakness, brain, mri, dementia, ct, e...  Nervous System
3  drop, glaucoma, vision, solution, occlusion, u...          Kidney
4  ckd, gfr, aki, eskd, dialysis, cyst, kidney, p...    Hypertension


In [7]:
def clean_text_func(text):
    
    """ this function clean & pre-process the data  """

    text = str(text)
    text = text.lower()
    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!?.\/'+]", " ", text)
    text = re.sub(r"\+", " ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ", text)
    text = re.sub(r"\?", " ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r"\s{2,}", " ", text)
    text = re.sub(r"[0-9]", " ", text)
    final_text = ""
    for x in text.split():
        if x not in domain_stop_word:
            final_text = final_text + x  +" "
    return final_text

df['Description'] = df['Description'].apply(lambda x: clean_text_func(x))
df.head()

,Description,D_Name
0,cancer tumor survival carcinoma prostate metas...,Cancer
1,artery seizure hg study kidney weakness brain ...,Diabetes
2,seizure weakness brain mri dementia ct epileps...,Nervous System
3,drop glaucoma vision solution occlusion uveiti...,Kidney
4,ckd gfr aki eskd dialysis cyst kidney plasma h...,Hypertension


# Words Embedding:

In [8]:
cv = CountVectorizer(stop_words="english")
cv_tfidf = TfidfVectorizer(stop_words="english")

X = cv.fit_transform(list(df.loc[:, 'Description' ]))
X_tfidf = cv_tfidf.fit_transform(list(df.loc[:, 'Description' ]))

In [9]:
df_cv = pd.DataFrame(X.toarray() , columns=cv.get_feature_names_out())
df_tfidf = pd.DataFrame(X_tfidf.toarray() , columns=cv_tfidf.get_feature_names_out())

In [10]:
print(df_cv.shape)
cosine = lambda v1 , v2 : dot(v1 , v2) / (norm(v1) * norm(v2))

(6, 96)


### Cosine Similarity 

In [11]:
new_text = ["medical health information"]
new_text_cv = cv.transform(new_text).toarray()[0]
new_text_tfidf = cv_tfidf.transform(new_text).toarray()[0]

for chpter_number in range(int(df.shape[0])):
    print(f"This is chpter number : {chpter_number} ")
    print(f"Cosin cv :    { cosine( df_cv.iloc[chpter_number]  , new_text_cv )} ")
    print(f"Cosin TFIDF : { cosine( df_tfidf.iloc[chpter_number]  , new_text_tfidf) } ")

This is chpter number : 0 
Cosin cv :    nan 
Cosin TFIDF : nan 
This is chpter number : 1 
Cosin cv :    nan 
Cosin TFIDF : nan 
This is chpter number : 2 
Cosin cv :    nan 
Cosin TFIDF : nan 
This is chpter number : 3 
Cosin cv :    nan 
Cosin TFIDF : nan 
This is chpter number : 4 
Cosin cv :    nan 
Cosin TFIDF : nan 
This is chpter number : 5 
Cosin cv :    nan 
Cosin TFIDF : nan 


C:\Users\HP\AppData\Local\Temp\ipykernel_12764\111292177.py:2: RuntimeWarning: invalid value encountered in scalar divide
  cosine = lambda v1 , v2 : dot(v1 , v2) / (norm(v1) * norm(v2))


### Implementing the classification model: LogisticRegression

In [12]:
df.columns

Index(['Description', 'D_Name'], dtype='object')

In [13]:
X_train = df.Description
y_train = df.D_Name

In [14]:
cv1 = CountVectorizer()
X_train_cv1 = cv1.fit_transform(X_train)
pd_cv1 = pd.DataFrame(X_train_cv1.toarray(), columns=cv1.get_feature_names_out())

In [15]:
lr = LogisticRegression()
lr.fit(X_train_cv1, y_train)

LogisticRegression()

In [37]:
X_test = "brain tumor"
cleaned_text = clean_text_func(X_test)

In [38]:
X_test_cv3  = cv1.transform([cleaned_text])
y_pred_cv3 = lr.predict(X_test_cv3)
print(y_pred_cv3)

['Nervous System']


### In conclusion our model is abel to predict the disease, that given by user